# nb

> Create FastHTML from a NB

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
%%html
<script type="module" src="https://cdn.jsdelivr.net/npm/zero-md@3?register"></script>

In [ ]:
#| export
from fasthtml.common import *
from pathlib import Path
import json

In [ ]:
example_nb_dir = Path('../example_nbs/')
with open(example_nb_dir/'explaining_xt_components.ipynb', 'r') as f: xt_nb = json.load(f)

In [ ]:
#| export
def strip_list(l, val='\n'):
    start, end = 0, len(l)
    while start < end and l[start] == val: start += 1
    while end > start and l[end - 1] == val: end -= 1
    return l[start:end]

In [ ]:
#| export
def render_md(c):
    # TODO default to FastHTML's implementation
    return c

In [ ]:
#| export
def render_md_cell(cell):
    assert cell['cell_type'] == 'markdown'
    return Div(cls='marked')(render_md(''.join(strip_list(cell['source']))))

In [ ]:
# show(Div(*map(render_md_cell,[o for o in xt_nb['cells'][:3] if o['cell_type']=='markdown'])))

In [ ]:
#| export
def get_nb_lang(nb): return nb['metadata']['kernelspec']['language']
# get_nb_lang(xt_nb)

In [ ]:
#| export
def render_code_source(cell,lang='python'):
    if cell['source']==[]: return ''
    code = f'''```{lang}\n{''.join(strip_list(cell['source']))}'''
    return Div(cls='marked')(render_md(code))

In [ ]:
# show(Div(*map(render_code_source,[o for o in xt_nb['cells'][:10] if o['cell_type']=='code'])))

In [ ]:
#| export
def render_code_output(cell,lang='python'):
    res = []
    if len(cell['outputs'])==0: ''
    for output in cell['outputs']:
        if output['output_type'] == 'execute_result':
            data = output['data']
            if 'text/markdown' in data.keys(): 
                res.append(NotStr(''.join(strip_list(data['text/markdown'][1:-1]))))
            elif 'text/plain' in data.keys(): 
                res.append(''.join(strip_list(data['text/plain'])))
        if output['output_type'] == 'stream':
            res.append(''.join(strip_list(output['text'])))
    return Footer(cls='marked')(*res)

In [ ]:
nbs_dir = Path('../example_nbs/')
with open(nbs_dir/'00_core.ipynb', 'r') as f: nb = json.load(f)

# render_code_output(nb['cells'][9])

In [ ]:
# show(Div(*map(render_code_output,[o for o in xt_nb['cells'][:20] if o['cell_type']=='code'])))

In [ ]:
#| export
def render_nb(fpath, # Path to Jupyter Notebook
              wrapper=Main, #Wraps entire rendered NB, default is for pico
              cls='container', # cls to be passed to wrapper, default is for pico
              md_cell_wrapper=Div, # Wraps markdown cell
              md_fn=render_md_cell, # md cell -> rendered html
              code_cell_wrapper=Card, # Wraps Source Code (body) + Outputs (footer)
              cd_fn=render_code_source, # code cell -> code source rendered html
              out_fn=render_code_output, # code cell -> code output rendered html
              **kwargs # Passed to wrapper
             ): 
    with open(fpath, 'r') as f: xt_nb = json.load(f)
    fname = Path(fpath).name
    res = []
    for cell in xt_nb['cells']:
        if cell['cell_type']=='code':
            s,o = cd_fn(cell), out_fn(cell)
            res.append(code_cell_wrapper(s,o))
        elif cell['cell_type']=='markdown': 
            res.append(md_cell_wrapper(md_fn(cell)))
    return wrapper(cls=cls)(*res)

## Todo

+ Quarto stuff rendered?
+ Implement default for Markdown based on FastHTML's default

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()